In [1]:
import torch
from torch import nn

import glob
import os
from tqdm import tqdm
from datetime import datetime
import json

from itertools import combinations

import torchvision
from torchvision.transforms import v2
from torchvision import tv_tensors
from torchvision import models

import segmentation_models_pytorch as smp

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

from sklearn.model_selection import train_test_split
from sklearn import metrics

import numpy as np

import pandas as pd

c:\Users\admin\miniconda3\envs\deep_learning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Объявление классов и функций

In [2]:
# datasets
class SegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, path_to_dataset_root:str, samples_df:pd.DataFrame, channel_indices:list, transforms:v2._transform.Transform, device:torch.device):
        '''
        In:
            path_to_dataset_root - путь до корневой папки с датасетом
            samples_df - pandas.DataFrame с информацией о файлах
            channel_indices - список с номерами каналов мультиспектрального изображения
            transforms - аугментация изображений
        '''
        super().__init__()
        self.path_to_dataset_root = path_to_dataset_root
        self.samples_df = samples_df
        self.channel_indices = channel_indices
        self.transforms = transforms
        self.device = device

    def __len__(self):
        return len(self.samples_df)

    def __getitem__(self, idx):
        sample = self.samples_df.iloc[idx]

        file_name = sample['file_name']

        path_to_image = os.path.join(self.path_to_dataset_root, 'images', f'{file_name}.npy')
        path_to_labels = os.path.join(self.path_to_dataset_root, 'labels', f'{file_name}.npy')

        image = torch.as_tensor(np.load(path_to_image), dtype=torch.int16)[self.channel_indices]
        #image = np.load(path_to_image)
        # метки читаем как одноканальное изображение
        label = np.load(path_to_labels)
        label = np.where(label >= 0, label, 0)
        #label = torch.as_tensor(np.load(path_to_labels), dtype=torch.uint8).long()
        label = torch.as_tensor(label, dtype=torch.uint8).long()
        
        
        image = tv_tensors.Image(image, device=self.device)
        label = tv_tensors.Mask(label, device=self.device)

        transforms_dict = {'image':image, 'mask':label}
        transformed = self.transforms(transforms_dict)
        return transformed['image'], transformed['mask']#, image
    
class SegmentationDatasetApplSurf(torch.utils.data.Dataset):
    def __init__(self, path_to_dataset_root, samples_df, channel_indices, name2class_idx_dict, applicable_surfaces_dict, transforms, device):
        super().__init__()
        self.path_to_dataset_root = path_to_dataset_root
        self.samples_df = samples_df
        self.channel_indices = channel_indices
        self.transforms = transforms
        self.device = device
        self.applicable_surfaces_dict = applicable_surfaces_dict
        self.idx2appl = {int(v): 'appl' if v else 'non_appl' for v in applicable_surfaces_dict.values()}
        self.applicable_indices = [name2class_idx_dict[cl] for cl, ap in applicable_surfaces_dict.items() if ap]


    def __len__(self):
        return len(self.samples_df)

    def __getitem__(self, idx):
        sample = self.samples_df.iloc[idx]

        file_name = sample['file_name']

        path_to_image = os.path.join(self.path_to_dataset_root, 'images', f'{file_name}.npy')
        path_to_labels = os.path.join(self.path_to_dataset_root, 'labels', f'{file_name}.npy')

        image = torch.as_tensor(np.load(path_to_image), dtype=torch.int16)[self.channel_indices]
        #image = np.load(path_to_image)
        # метки читаем как одноканальное изображение
        label = torch.as_tensor(np.load(path_to_labels), dtype=torch.uint8).long()

        applicable_filter = label == self.applicable_indices[0]
        for appl_i in self.applicable_indices[1:]:
            applicable_filter = applicable_filter | (label == appl_i)
        applicable_label = torch.where(applicable_filter==True, 1, 0)
        
        
        image = tv_tensors.Image(image, device=self.device)
        applicable_label = tv_tensors.Mask(applicable_label, device=self.device)

        transforms_dict = {'image':image, 'mask':applicable_label}
        transformed = self.transforms(transforms_dict)
        return transformed['image'], transformed['mask']#, image
    
class FCNSegmentationWrapper(nn.Module):
    def __init__(self, model:nn.Module):
        '''
        Обертка для модели FCN из библиотеки pytorch
        In:
            model - нейронная сеть
        '''
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)['out']
    
class MultispectralNN(nn.Module):
    def __init__(self, main_model:nn.Module, preprocessing_block:nn.Module):
        '''
        Нейронная сеть с блоком предварительной обработки каналов мультиспектра
        In:
            main_model - нейронная сеть
            preprocessing_block - блок предобработки каналов мультиспектра
        '''
        super().__init__()
        self.preprocessing_block = preprocessing_block
        self.main_model = main_model

    def forward(self, x):
        x = self.preprocessing_block(x)
        return self.main_model(x)
    
class MultispectralNNOutProcess(nn.Module):
    def __init__(self, main_model:nn.Module, preprocessing_block:nn.Module, out_block:nn.Module):
        '''
        Нейронная сеть с блоками пред- и постобработки
        In:
            main_model - нейронная сеть
            preprocessing_block - блок предобработки каналов мультиспектра
            out_block - блок постобработки
        '''
        super().__init__()
        self.preprocessing_block = preprocessing_block
        self.main_model = main_model
        self.out_block = out_block

    def forward(self, x):
        preprocessed = self.preprocessing_block(x)
        result = self.main_model(x)
        return self.out_block(result, preprocessed)
    
class MultispectralDataOutput(nn.Module):
    def __init__(self, in_channels, class_num):
        super().__init__()
        self.low_level_conv = nn.Conv1d(in_channels, class_num, kernel_size=1)
        self.out_conv = nn.Conv1d(class_num*2, class_num, kernel_size=1)
    def forward(self, cnn_out, low_level_out):
        low_level_results = self.low_level_conv(low_level_out)
        nn.ChannelShuffle(groups=2)

class MultispectralFuseOut(nn.Module):
    def __init__(self, main_model:nn.Module, multispectral_preprocessing_block:nn.Module, fusion_type:str, preprocessing_out_dim:int, class_num:int):
        '''
        In:
            main_model - нейронная сеть
            multispectral_preprocessing_block - блок предобработки каналов мультиспектра
            fusion_type - строка с типом слияния  ('shuffle', 'concat', 'add')
            preprocessing_out_dim - целое число, выражающее разерность блока предобрботки каналов мультиспектра
            class_num: кол-во классов
        '''
        super().__init__()
        self.multispectral_preprocessing_block = multispectral_preprocessing_block
        self.main_model = main_model

        self.multispectral_preout_block = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channels=preprocessing_out_dim,out_channels=class_num, kernel_size=1),
            nn.BatchNorm2d(class_num),
            nn.ReLU()
        )
        self.fusion_type = fusion_type
        if fusion_type == 'shuffle':
            self.fusion_block = nn.Sequential(
                #nn.Dropout2d(0.3),
                nn.ChannelShuffle(groups=2),
                nn.Conv2d(in_channels=class_num*2, out_channels=class_num, kernel_size=1, groups=class_num)
            )
        elif fusion_type == 'concat':
            self.fusion_block = nn.Conv2d(in_channels=class_num*2, out_channels=class_num, kernel_size=1)
        elif fusion_type == 'add':
            self.fusion_block = nn.Conv2d(in_channels=class_num, out_channels=class_num, kernel_size=1)

    def forward(self, x):
        multispectral_preprocessed_out = self.multispectral_preprocessing_block(x)
        multispectral_out = self.multispectral_preout_block(multispectral_preprocessed_out)
        #print(multispectral_preprocessed_out.shape)
        #print(multispectral_out.shape)
        main_out = self.main_model(multispectral_preprocessed_out)
        if self.fusion_type == 'add':
            concat_out = multispectral_out + main_out
        else:
            concat_out = torch.cat([multispectral_out, main_out], dim=1)
        

        return self.fusion_block(concat_out)

class MultitaskLoss(nn.Module):
    def __init__(self, loss1, loss2):
        super().__init__()
        self.loss1 = loss1
        self.loss2 = loss2
        
    def forward(self, pred1, pred2, target1, target2):
        loss_val1 = self.loss1(pred1, target1)
        loss_val2 = self.loss1(pred2, target2)
        return loss_val1 + loss_val2
        #applicable_target = 
        #coarse_target = 

class MultitaskModel(nn.Module):
    def __init__(self, model, nn_output_size, appl_class_num, surf_class_num):
        super().__init__()
        self.model = model
        self.applicable_head = nn.Sequential(
            #nn.Conv2d(nn_output_size, out_channels=nn_output_size//4, kernel_size=3, padding=1),
            #nn.BatchNorm2d(nn_output_size//4),
            #nn.Dropout2d(p=0.3),
            #nn.Conv2d(nn_output_size//4, appl_class_num, kernel_size=1)
            nn.Conv2d(nn_output_size, appl_class_num, kernel_size=1)
        )
        self.surf_head = nn.Sequential(
            #nn.Conv2d(nn_output_size, out_channels=nn_output_size//4, kernel_size=3, padding=1),
            #nn.BatchNorm2d(nn_output_size//4),
            #nn.Dropout2d(p=0.3),
            #nn.Conv2d(nn_output_size//4, surf_class_num, kernel_size=1)
            nn.Conv2d(nn_output_size, surf_class_num, kernel_size=1)
        )

    def forward(self,x):
        h = self.model(x)
        appl_out = self.applicable_head(h)
        surf_out = self.surf_head(h)
        return appl_out, surf_out

class SpectralDiffIndexModule(nn.Module):
    def __init__(self, channel_indices_list, channels_in_index, out_channels):
        super().__init__()
        self.channel_indices_list = channel_indices_list
        combinations_list = list(combinations(channel_indices_list, channels_in_index))
        
        self.combinations_list = np.array(combinations_list).reshape(-1).tolist()
        in_channels = len(self.combinations_list)
        #self.make_channels_combinations = MakeChannelsCombinations(self.combinations_list)
        self.numerator = nn.Conv2d(in_channels=in_channels, out_channels=in_channels//channels_in_index, kernel_size=1, groups=in_channels//channels_in_index, bias=False)
        self.denominator = nn.Conv2d(in_channels=in_channels, out_channels=in_channels//channels_in_index, kernel_size=1, groups=in_channels//channels_in_index, bias=False)
        self.out_block = nn.Sequential(
            nn.Conv2d(in_channels=in_channels//channels_in_index, out_channels=out_channels, kernel_size=1),
            nn.BatchNorm2d(out_channels),
            #nn.ReLU()
        )

    def forward(self, x):
        channels_combinations = x[:,self.combinations_list]
        numerator_results = self.numerator(channels_combinations)
        denominator_results = self.denominator(channels_combinations)
        indices = numerator_results / (denominator_results+1e-7)
        output = self.out_block(indices)
        return output

def decode_confusion_matrix_2x2(confusion_matrix):
    '''
    In: Матрица ошибок со следующей структурой:
    [[tn, fp],
     [fn, tp]]
    Out: словарь {'tp':tp_val, 'tn': tn_val, 'fp': fp_val, 'fn': fn_val}
    '''
    tp = confusion_matrix[1, 1]
    tn = confusion_matrix[0, 0]
    fp = confusion_matrix[0, 1]
    fn = confusion_matrix[1, 0]
    return {'tp':tp, 'tn': tn, 'fp': fp, 'fn': fn}

def compute_accuracy_from_confusion(multiclass_confusion_matrix):
    '''
    In:
        multiclass_confusion_matrix - np.ndarray размера (class_num, 2, 2)
    Out:
        accuracy - float результат вычисления точности
    '''

    confusion_sum = multiclass_confusion_matrix.sum(axis=0)
    tp, tn, fp, fn = decode_confusion_matrix_2x2(confusion_sum)
    accuracy = 0
    if tp+tn+fp+fn != 0:
        accuracy = (tp+tn)/(tp+tn+fp+fn)
    return accuracy

def compute_metric_from_confusion(multiclass_confusion_matrix:np.ndarray, metric_params_dict:dict, idx2class_name_dict=None):
    '''
    Вычисление значений метрик, вычисляющитхся для одного класса (precsion, recall, f1-score, IoU, Dice)
    Вычисляет как метрики для всех классов, так и невзвешенное среднее значение метрики для всех классов
    In:
        multiclass_confusion_matrix - многоклассовая матрица ошибок
        metric_params_dict - словарь c информацией о том, как вычислять метрики. Cловарь метрик имеет след. структуру:
            {'name': 'имя_метрики', 'numerator': [список со строковыми обозначениями 'tp', 'fp', 'tn', 'fn'], 'denominator': [список со строковыми обозначениями 'tp', 'fp', 'tn', 'fn']}
            Списки 'numerator' и 'denominator' зависят от формулы вычисления метрики. Например, precision=tp/(tp+fp)
            Тогда numerator: ['tp'], denominator: ['tp', 'fp']
            Метрики вычисляются по общей формуле metric_val = sum(numerator_vals_list)/sum(denominators_vals_list)
    Out:
        metric_dict - словарь со структурой {metric_name: metric_val}
    '''
    #print(f'metric_params_dict={metric_params_dict}')
    mean_metric = 0
    # {class_name: iou_val}
    metric_dict = {}
    actual_classes_num = 0
    metric_name = metric_params_dict['name']
    # отдельная обработка метрики accuracy, которая считается для всех классов сразу
    if metric_name == 'accuracy':
        # получаем матрицу ошибок для всех классов сразу
        confusion_sum = multiclass_confusion_matrix.sum(axis=0)
        # выделяем словарь, содержащий общее количество распознаваний {'tp': tp_num, 'fp': fp_num, 'tn': tn_num, 'fn': fn_num}
        confusion_vals_dict = decode_confusion_matrix_2x2(confusion_sum)
        # выделение списка значений 'tp', 'fp', 'tn', 'fn', входящих в числитель формулы метрики
        numerator = [confusion_vals_dict[v] for v in metric_params_dict['numerator']]
        # выделение списка значений ('tp', 'fp', 'tn', 'fn'), входящих в знаменатель формулы метрики
        denominator = [confusion_vals_dict[v] for v in metric_params_dict['denominator']]
        accuracy = 0
        if np.sum(denominator) != 0:
            class_metric = np.sum(numerator)/np.sum(denominator)
        metric_dict[metric_name] = class_metric
        return metric_dict

    for idx, class_confusion in enumerate(multiclass_confusion_matrix):
        # выделяем словарь, содержащий количество распознаваний {'tp': tp_num, 'fp': fp_num, 'tn': tn_num, 'fn': fn_num} для определенного класса
        confusion_vals_dict = decode_confusion_matrix_2x2(class_confusion)
        
        if class_confusion.sum() != confusion_vals_dict['tn']:
            actual_classes_num += 1

        #print(f'actual_classes_num={actual_classes_num}')
        class_metric = 0
        # выделение списка значений 'tp', 'fp', 'tn', 'fn', входящих в числитель формулы метрики
        numerator = [confusion_vals_dict[v] for v in metric_params_dict['numerator']]
        # выделение списка значений 'tp', 'fp', 'tn', 'fn', входящих в знаменатель формулы метрики
        denominator = [confusion_vals_dict[v] for v in metric_params_dict['denominator']]
        
        if np.sum(denominator) != 0:
            class_metric = np.sum(numerator)/np.sum(denominator)
        # вычисление суммарного значения метрики для дальнейшего вычисления среднего згначения по всем классам
        mean_metric += class_metric
        class_name = f'{metric_name}_{idx}'
        if idx2class_name_dict is not None:
            class_name = f'{metric_name}_{idx2class_name_dict[idx]}'
        metric_dict[class_name] = class_metric
    if actual_classes_num == 0:
        metric_dict[f'{metric_name}_mean'] = 0
    else:
        # вычисление среднего значения метрики
        metric_dict[f'{metric_name}_mean'] = mean_metric/actual_classes_num
    return metric_dict

def compute_pred_mask(pred):
    '''
    Определение маски классов на основе сгенерированной softmax маски
    '''
    pred = pred.detach()
    _, pred_mask = pred.max(dim=1)
    return pred_mask.cpu().numpy()

class SegmentationModule(L.LightningModule):
    def __init__(self, model:nn.Module, criterion:nn.Module, metrics_info_list:list, name2class_idx_dict:dict) -> None:
        '''
        Модуль Lightning для обучения сегментационной сети
        In:
            model - нейронная сеть
            criterion - функция потерь
            metrics_info_list - список со словарями метрик. Каждый словарь метрик имеет след. структуру:
                {'name': 'имя_метрики', 'numerator': [список со строковыми обозначениями 'tp', 'fp', 'tn', 'fn'], 'denominator': [список со строковыми обозначениями 'tp', 'fp', 'tn', 'fn']}
                Списки 'numerator' и 'denominator' зависят от формулы вычисления метрики. Например, precision=tp/(tp+fp)
                Тогда numerator: ['tp'], denominator: ['tp', 'fp']
            name2class_idx_dict - словарь с отображением {class_name(str): class_idx(int)}
        '''
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.name2class_idx_dict = name2class_idx_dict
        # словарь, выполняющий обратное отображение class_idx в class_name
        self.class_idx2name_dict = {v:k for k, v in name2class_idx_dict.items()}
        # train_confusion_matrix - матрица ошибок для вычисления метрик на этапе обучения. Размер (class_num, 2, 2) должен соответствовать размеру, генериуемому функцией sklearn.metrics.multilabel_confusion_matrix
        self.train_confusion_matrix = np.zeros((len(self.name2class_idx_dict), 2, 2), dtype=np.int64)
        # val_confusion_matrix - матрица ошибок для вычисления метрик на этапе тестирования. Размер (class_num, 2, 2) должен соответствовать размеру, генериуемому функцией sklearn.metrics.multilabel_confusion_matrix
        self.val_confusion_matrix = np.zeros((len(self.name2class_idx_dict), 2, 2), dtype=np.int64)
        self.metrics_info_list = metrics_info_list
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())
    
    def training_step(self, batch, batch_idx):
        data, true_labels = batch
        pred = self.model(data)
        loss = self.criterion(pred, true_labels)
        # вычисление сгенерированной маски
        pred_labels = compute_pred_mask(pred)
        true_labels = true_labels.detach().cpu().numpy()
        # вычисление многоклассовой матрицы ошибок для одного пакета (batch) обработанных данных
        batch_confusion_matrix = metrics.multilabel_confusion_matrix(true_labels.reshape(-1), pred_labels.reshape(-1),labels=list(self.class_idx2name_dict.keys()))
        #print(f'train_batch_conf_type={batch_confusion_matrix.dtype}')
        #print(batch_confusion_matrix)
        # приплюсовываем результаты на батче к результатам на эпохе
        self.train_confusion_matrix += batch_confusion_matrix.astype(np.int64)
        # т.к. мы вычисляем общую ошибку на всей эпохе, то записываем в лог только значение функции потерь
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        data, true_labels = batch
        pred = self.model(data)
        loss = self.criterion(pred, true_labels)
        pred_labels = compute_pred_mask(pred)
        true_labels = true_labels.detach().cpu().numpy()
        batch_confusion_matrix = metrics.multilabel_confusion_matrix(true_labels.reshape(-1), pred_labels.reshape(-1),labels=list(self.class_idx2name_dict.keys()))
        #print(f'pred_labels type={pred_labels.dtype}')
        #print(f'true_labels type={true_labels.dtype}')
        #print(f'val_batch_conf_type={batch_confusion_matrix.dtype}')
        #print(batch_confusion_matrix)
        self.val_confusion_matrix += batch_confusion_matrix.astype(np.int64)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss
    
    def on_train_epoch_end(self):
        '''
        Декодирование результатов тренировочной эпохи и запись их в лог
        '''
        # итерируем по рассматриваемым метрикам (структуру self.metrics_info_list см. в конструкторе)
        for metric_info_dict in self.metrics_info_list:
            metric_dict = compute_metric_from_confusion(self.train_confusion_matrix, metric_info_dict, self.class_idx2name_dict)
            for name, value in metric_dict.items():
                name = f'train_{name}'
                self.log(name, value, on_step=False, on_epoch=True, prog_bar=True)
        self.train_confusion_matrix = np.zeros((len(self.name2class_idx_dict), 2, 2), dtype=np.int64)
        
    def on_validation_epoch_end(self):
        '''
        Декодирование результатов тестовой эпохи и запись их в лог
        (работает точно также, как и )
        '''
        for metric_info_dict in self.metrics_info_list:
            metric_dict = compute_metric_from_confusion(self.val_confusion_matrix, metric_info_dict, self.class_idx2name_dict)
            for name, value in metric_dict.items():
                name = f'val_{name}'
                self.log(name, value, on_step=False, on_epoch=True, prog_bar=True)
        self.val_confusion_matrix = np.zeros((len(self.name2class_idx_dict), 2, 2), dtype=np.int64)

class MultitaskSegmentationModule(L.LightningModule):
    def __init__(self, model, criterion, metrics_info_list, name2class_idx_dict, applicable_surfaces_dict) -> None:
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.name2class_idx_dict = name2class_idx_dict
        self.class_idx2name_dict = {v:k for k, v in name2class_idx_dict.items()}
        self.idx2appl = {int(v): 'appl' if v else 'non_appl' for v in applicable_surfaces_dict.values()}
        self.applicable_indices = [name2class_idx_dict[cl] for cl, ap in applicable_surfaces_dict.items() if ap]
        self.classes_train_confusion_matrix = np.zeros((len(self.name2class_idx_dict), 2, 2), dtype=np.int64)
        self.classes_val_confusion_matrix = np.zeros((len(self.name2class_idx_dict), 2, 2), dtype=np.int64)
        self.applicable_train_confusion_matrix = np.zeros((len(self.idx2appl), 2, 2), dtype=np.int64)
        self.applicable_val_confusion_matrix = np.zeros((len(self.idx2appl), 2, 2), dtype=np.int64)
        self.metrics_info_list = metrics_info_list
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())
    
    def training_step(self, batch, batch_idx):
        data, classes_true_labels = batch

        applicable_filter = classes_true_labels == self.applicable_indices[0]
        for appl_i in self.applicable_indices[1:]:
            applicable_filter = applicable_filter | (classes_true_labels == appl_i)
        applicable_true_labels = torch.where(applicable_filter==True, 1, 0)

        applicable_pred, classes_pred = self.model(data)

        loss = self.criterion(applicable_pred, classes_pred, applicable_true_labels, classes_true_labels)
        
        applicable_pred_labels = compute_pred_mask(applicable_pred)
        classes_pred_labels = compute_pred_mask(classes_pred)
        applicable_true_labels = applicable_true_labels.detach().cpu().numpy()
        classes_true_labels = classes_true_labels.detach().cpu().numpy()
        classes_batch_confusion_matrix = metrics.multilabel_confusion_matrix(classes_true_labels.reshape(-1), classes_pred_labels.reshape(-1),labels=list(self.class_idx2name_dict.keys()))
        applicable_batch_confusion_matrix = metrics.multilabel_confusion_matrix(applicable_true_labels.reshape(-1), applicable_pred_labels.reshape(-1),labels=[0, 1])
        #print(f'train_batch_conf_type={batch_confusion_matrix.dtype}')
        #print(batch_confusion_matrix)
        self.classes_train_confusion_matrix += classes_batch_confusion_matrix.astype(np.int64)
        self.applicable_train_confusion_matrix += applicable_batch_confusion_matrix.astype(np.int64)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        data, classes_true_labels = batch

        applicable_filter = classes_true_labels == self.applicable_indices[0]
        for appl_i in self.applicable_indices[1:]:
            applicable_filter = applicable_filter | (classes_true_labels == appl_i)
        applicable_true_labels = torch.where(applicable_filter==True, 1, 0)

        applicable_pred, classes_pred = self.model(data)

        loss = self.criterion(applicable_pred, classes_pred, applicable_true_labels, classes_true_labels)
        
        applicable_pred_labels = compute_pred_mask(applicable_pred)
        classes_pred_labels = compute_pred_mask(classes_pred)
        applicable_true_labels = applicable_true_labels.detach().cpu().numpy()
        classes_true_labels = classes_true_labels.detach().cpu().numpy()
        classes_batch_confusion_matrix = metrics.multilabel_confusion_matrix(classes_true_labels.reshape(-1), classes_pred_labels.reshape(-1),labels=list(self.class_idx2name_dict.keys()))
        applicable_batch_confusion_matrix = metrics.multilabel_confusion_matrix(applicable_true_labels.reshape(-1), applicable_pred_labels.reshape(-1),labels=[0, 1])
        #print(f'train_batch_conf_type={batch_confusion_matrix.dtype}')
        #print(batch_confusion_matrix)
        self.classes_val_confusion_matrix += classes_batch_confusion_matrix.astype(np.int64)
        self.applicable_val_confusion_matrix += applicable_batch_confusion_matrix.astype(np.int64)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss
    
    def on_train_epoch_end(self):
        for metric_info_dict in self.metrics_info_list:
            metric_name = metric_info_dict['name']
            class_metric_dict = compute_metric_from_confusion(self.classes_train_confusion_matrix, metric_info_dict, self.class_idx2name_dict)
            mean = 0
            for name, value in class_metric_dict.items():
                if 'mean' in name:
                    class_mean = value
                name = f'tr_cl_{name}'
                self.log(name, value, on_step=False, on_epoch=True, prog_bar=True)
            appl_metric_dict = compute_metric_from_confusion(self.applicable_train_confusion_matrix, metric_info_dict, self.idx2appl)
            for name, value in appl_metric_dict.items():
                if 'mean' in name:
                    appl_mean = value
                name = f'tr_ap_{name}'
                self.log(name, value, on_step=False, on_epoch=True, prog_bar=True)
            mean = (class_mean+appl_mean)/2
            self.log(f'tr_{metric_name}_mean', mean, on_step=False, on_epoch=True, prog_bar=True)
        self.classes_train_confusion_matrix = np.zeros((len(self.name2class_idx_dict), 2, 2), dtype=np.int64)
        self.applicable_train_confusion_matrix = np.zeros((len(self.idx2appl), 2, 2), dtype=np.int64)
        
    def on_validation_epoch_end(self):
        for metric_info_dict in self.metrics_info_list:
            metric_name = metric_info_dict['name']
            class_metric_dict = compute_metric_from_confusion(self.classes_val_confusion_matrix, metric_info_dict, self.class_idx2name_dict)
            mean = 0
            for name, value in class_metric_dict.items():
                if 'mean' in name:
                    class_mean = value
                name = f'v_cl_{name}'
                self.log(name, value, on_step=False, on_epoch=True, prog_bar=True)
            appl_metric_dict = compute_metric_from_confusion(self.applicable_val_confusion_matrix, metric_info_dict, self.idx2appl)
            for name, value in appl_metric_dict.items():
                if 'mean' in name:
                    appl_mean = value
                name = f'v_ap_{name}'
                self.log(name, value, on_step=False, on_epoch=True, prog_bar=True)
            mean = (class_mean+appl_mean)/2
            self.log(f'v_{metric_name}_mean', mean, on_step=False, on_epoch=True, prog_bar=True)
        self.classes_val_confusion_matrix = np.zeros((len(self.name2class_idx_dict), 2, 2), dtype=np.int64)
        self.applicable_val_confusion_matrix = np.zeros((len(self.idx2appl), 2, 2), dtype=np.int64)

def prepare_channel_indices_str(channel_indices_list):
    '''
    Функция формирует строку, содержащую обрабатываемые индексы мультиспектра. Нужна для формирования красивого имени сохраняемого файла
    '''
    last = None
    first = None
    intervals = []
    for i, ch_idx in enumerate(sorted(channel_indices_list)):
        #print(f'ch_idx={ch_idx}, last==ch_idx-1:{last==ch_idx-1}')
        #print(f'first={first};last={last}')
        if last is not None:
            if last != ch_idx-1:
                #last = prev_idx
                if first == last:
                    intervals.append([last])
                else:
                    intervals.append([first, last])

                first = ch_idx
                last = ch_idx
            else:
                last = ch_idx
            if i == len(channel_indices_list)-1:
                if first == last:
                    intervals.append([last])
                else:
                    intervals.append([first, last])
        else:
            first = ch_idx
            last = ch_idx

        #channels_str += f'{ch},'
    chanels_str = ''
    for i, interval in enumerate(intervals):
        if len(interval) == 1:
            ptr = f'{interval[0]}'
        else:
            ptr = f'{interval[0]}-{interval[-1]}'
        if i != len(intervals) - 1:
            ptr = f'{ptr},'

        chanels_str += ptr
    return chanels_str

# Функции-фабрики для генерирования различных конфигураций нейронных сетей

In [10]:
def prepare_torch_model(
        model_dict:dict,
        image_channels:int,
        nn_in_cannels_num:int,
        class_num:int,
        channel_indices_list:list,
        is_multitask:bool,
        preprocess_params:dict,
        fuze_params:dict):
    '''
    Функция генерирует конфигурацию сегментационной нейронной сети из библиотеки torchvision.models.segmentation
    In:
        model_dict - словарь с параметрами модели. Структура словаря {'model_name': строковое_имя_модели, 'creation_function': функция_создания_модели (класс модели), 'weights': веса модели}
        image_channels:int - количество каналов изображения
        nn_in_cannels_num:int - количество входных каналов нейронной сети
        class_num:int - количество классов
        channel_indices_list:list - список индексов каналов обрабатываемого мультиспектрального изображения
        is_multitask:bool - флаг, обозначающий мнгогозадачное обучение
        preprocess_params:dict - параметры предварительной обработки каналов мультиспектра. Словарь структуры {'type':'1L'}, на месте '1L' мб
            'no' - отсутствие доп. блока предобработки,
            1L - однослойная модель предобработки
            2L - двухслойная модель предобработки
            SpInd - слой, вычисляющий множество спектральных яркостных разностных индексов
        fuze_params:dict - параметры связывания выхода нейронной сети и блока преобработки. Словарь структуры {'type':'no'}, на месте 'no' мб
            shuffle - модуль channel_shuffle (см. https://arxiv.org/abs/1707.01083)
            concat - конкатенация
            add - сложение
    Out:
        dict с моделью следующей структуры {'name': model_name, 'model': model, 'train_transforms':train_transforms, 'test_transforms':test_transforms}
    '''
    model_creation_unction = model_dict['creation_function']
    weights = model_dict['weights']
    model_name = model_dict['model_name']
    model = model_creation_unction(weights=weights)
    
    # заменяем входной слой, если количество входных каналов мультиспектра не равно трем
    conv1 = model.backbone.conv1

    weights = conv1.weight
    # если количество входных каналов не равно трем, то мы 
    # 1. Усредняем величины весов трех входных каналов предобученной нейронной сети
    # 2. Копируем полученные усредненные веса для всех входных каналов нового слоя 
    new_weight = torch.cat([weights.mean(dim=1).unsqueeze(1)]*nn_in_cannels_num, dim=1)
    # создаем новый слой
    new_conv1 = nn.Conv2d(
        in_channels=nn_in_cannels_num,
        out_channels=conv1.out_channels,
        kernel_size=conv1.kernel_size,
        stride=conv1.stride,
        padding=conv1.padding,
        dilation=conv1.dilation,
        groups=conv1.groups,
        bias=conv1.bias is not None
    )

    new_conv1.weight = nn.Parameter(new_weight)
    if conv1.bias is not None:
        # заменяем пороги, если они были в изначальном входном слое
        new_conv1.bias = model.backbone.conv1.bias
    # заменяем веса, если количнество входных каналов не равно трем 
    if nn_in_cannels_num != 3:
        model.backbone.conv1 = new_conv1

    # заменяем выходные слои, если количество каналов изображения не равно трем
    # 
    if is_multitask:
        model.classifier = nn.Sequential(*list(model.classifier.children())[:-1])
    else:
        classifier_conv = model.classifier[-1]
        new_classifier_conv = nn.Conv2d(
            in_channels=classifier_conv.in_channels,
            out_channels=class_num,
            kernel_size=classifier_conv.kernel_size,
            stride=classifier_conv.kernel_size,
            padding=classifier_conv.padding,
            dilation=classifier_conv.dilation,
            groups=classifier_conv.groups,
            bias=classifier_conv.bias is not None,
            )
        model.classifier[-1] = new_classifier_conv
    
    # выбор модели доп классификатора для модели из torchvision
    if 'fcn' in model_name.lower():
        model.classifier = models.segmentation.fcn.FCNHead(in_channels=2048, channels=class_num)
    elif 'dlv3' in model_name.lower():
        model.classifier = models.segmentation.deeplabv3.DeepLabHead(in_channels=2048, num_classes=class_num)

    # нам нужна обертка модели, чтобы для того, чтобы у нее был один выход (в моделях torchvision.models.segmentation по умолчанию два выхода - сегментационный и доп. классификатор)
    model = FCNSegmentationWrapper(model)
    if is_multitask:
        model = MultitaskModel(model, nn_output_size=512, appl_class_num=2, surf_class_num=class_num)

    # добавление дополнительных блоков предобработки каналов мультиспектра, если они есть 
    if preprocess_params['type'] == 'no':
        preprocess_layer = nn.Identity()
    elif preprocess_params['type'] == '1L':
        # создаем один сверточный слой с ядром размером 1x1
        # активация отсутствует, т.к. наличие активации показало худшие результаты
        preprocess_layer = nn.Sequential(
            nn.Conv2d(in_channels=image_channels, out_channels=nn_in_cannels_num, kernel_size=1),
            nn.BatchNorm2d(nn_in_cannels_num)
        )
    elif preprocess_params['type'] == '2L':
        # создаем двухслойную сверточную нейронную сеть с ядрами размером 1x1
        preprocess_layer = nn.Sequential(
            nn.Conv2d(in_channels=image_channels, out_channels=32, kernel_size=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=nn_in_cannels_num, kernel_size=1),
            nn.BatchNorm2d(nn_in_cannels_num),
            nn.ReLU())
    elif preprocess_params['type'] == 'SpInd':
        # слой, вычисляющий множество спектральных яркостных разностных индексов
        preprocess_layer = SpectralDiffIndexModule(channel_indices_list=channel_indices_list, channels_in_index=2, out_channels=8)
    
    if fuze_params['type'] == 'no':
        model = MultispectralNN(model, preprocess_layer)
    else:
        # добавляем слой, выполняющий слияние выхода нейронной сети и результатов блока предобработки мультиспектральных изображений
        model = MultispectralFuseOut(model, preprocess_layer, preprocessing_out_dim=nn_in_cannels_num, fusion_type=fuze_params['type'], class_num=class_num)

    # добавление аугментаций
    train_transforms = v2.Compose([v2.ToDtype(torch.float32, scale=True)])
    test_transforms = v2.Compose([v2.ToDtype(torch.float32, scale=True)])

    multitask_str = '_MT' if is_multitask else ''
    # формирование имени модели для сохранения в лог 
    model_name = f'{model_name}pr'
    if preprocess_params['type'] != 'no':
        model_name += f'-P{preprocess_params["type"]}'
    if fuze_params['type'] != 'no':
        model_name += f'-Fuz{fuze_params["type"].capitalize()}'
    
    #model_name = f'{model_name}pr-P2L-FuzOutAdd({nn_in_cannels_num})' + multitask_str
    
    #model_name = f'{model_name}pr' + multitask_str

    return {'name': model_name, 'model': model, 'train_transforms':train_transforms, 'test_transforms':test_transforms}

def prepare_smp_model(
        model_dict,
        image_channels,
        nn_in_cannels_num,
        class_num,
        channel_indices_list,
        is_multitask,
        preprocess_params,
        fuze_params):
    '''
    Функция генерирует конфигурацию сегментационной нейронной сети из библиотеки segmentation_models_pytorch
    In:
        model_dict - словарь с параметрами модели. Структура словаря {'model_name': строковое_имя_модели, 'creation_function': функция_создания_модели (класс модели), 'weights': веса модели}
        image_channels:int - количество каналов изображения
        nn_in_cannels_num:int - количество входных каналов нейронной сети
        class_num:int - количество классов
        channel_indices_list:list - список индексов каналов обрабатываемого мультиспектрального изображения
        is_multitask:bool - флаг, обозначающий мнгогозадачное обучение
        preprocess_params:dict - параметры предварительной обработки каналов мультиспектра. Словарь структуры {'type':'1L'}, на месте '1L' мб
            'no' - отсутствие доп. блока предобработки,
            1L - однослойная модель предобработки
            2L - двухслойная модель предобработки
            SpInd - SpatialIndex??? (надо подробнее рассмотреть это потом)
        fuze_params:dict - параметры связывания выхода нейронной сети и блока преобработки. Словарь структуры {'type':'no'}, на месте 'no' мб
            shuffle - модуль channel_shuffle (см. https://arxiv.org/abs/1707.01083)
            concat - конкатенация
            add - сложение
    Out:
        dict с моделью следующей структуры {'name': model_name, 'model': model, 'train_transforms':train_transforms, 'test_transforms':test_transforms}
    '''
    model_creation_unction = model_dict['creation_function']
    encoder_name = model_dict['encoder_name']
    model_name = model_dict['model_name']
    #model = model_creation_unction(encoder_name=encoder_name, encoder_weights='imagenet', encoder_depth=5, decoder_channels=(256, 128, 64, 64, 64), in_channels=nn_in_cannels_num, classes=class_num)
    model = model_creation_unction(encoder_name=encoder_name, encoder_weights=None, in_channels=nn_in_cannels_num, classes=class_num)
    #model = model_creation_unction(encoder_name=encoder_name, in_channels=nn_in_cannels_num, classes=class_num, img_size=112)

    #print(model)
    
    #model = smp.Unet(encoder_name='resnet50', classes=class_num)
    #print(model.encoder)
    #conv1 = model.encoder.model.conv1
    #conv1 = model.encoder.model.conv_stem
    conv1 = model.encoder._conv_stem
    #conv1 = model.encoder.model.patch_embed.proj
    
    '''
    weights = conv1.weight
    new_weight = torch.cat([weights.mean(dim=1).unsqueeze(1)]*nn_in_cannels_num, dim=1)
    new_conv1 = nn.Conv2d(
        in_channels=nn_in_cannels_num,
        out_channels=conv1.out_channels,
        kernel_size=conv1.kernel_size,
        #stride=conv1.stride,
        stride=(1,1),
        padding=conv1.padding,
        dilation=conv1.dilation,
        groups=conv1.groups,
        bias=conv1.bias is not None
    )
    new_conv1.weight = nn.Parameter(new_weight)
    if conv1.bias is not None:
        #new_conv1.bias = model.encoder.conv1.bias
        #new_conv1.bias = model.encoder.model.patch_embed.proj.bias
        new_conv1.bias = model.encoder.model.conv_stem.bias
    # заменяем веса, если количнество входных каналов не равно трем 
    if nn_in_cannels_num != 3:
        #model.encoder.conv1 = new_conv1
        #model.encoder.model.conv_stem = new_conv1
        model.encoder._conv_stem = new_conv1
        #model.encoder.model.patch_embed.proj = new_conv1
    '''
    # делаем два входных слоя для более глубокой обработки низкоуровневых призников
    conv1 = model.encoder._conv_stem
    weights = conv1.weight
    new_weight1 = torch.cat([weights.mean(dim=1).unsqueeze(1)]*nn_in_cannels_num, dim=1)
    new_weight2 = torch.cat([weights.mean(dim=1).unsqueeze(1)]*conv1.out_channels, dim=1)
    new_conv1 = nn.Conv2d(
        in_channels=nn_in_cannels_num,
        out_channels=conv1.out_channels,
        kernel_size=conv1.kernel_size,
        #stride=conv1.stride,
        stride=(1,1),
        #padding=conv1.padding,
        padding=(1, 1),
        dilation=conv1.dilation,
        groups=conv1.groups,
        bias=conv1.bias is not None
    )

    new_conv2 = nn.Conv2d(
        in_channels=conv1.out_channels,
        out_channels=conv1.out_channels,
        kernel_size=conv1.kernel_size,
        #stride=conv1.stride,
        stride=(1,1),
        padding=(1,1),
        dilation=conv1.dilation,
        groups=conv1.groups,
        bias=conv1.bias is not None
    )
        
    new_conv1.weight = nn.Parameter(new_weight1)
    new_conv2.weight = nn.Parameter(new_weight2)
    if conv1.bias is not None:
        #new_conv1.bias = model.encoder.conv1.bias
        #new_conv1.bias = model.encoder.model.patch_embed.proj.bias
        new_conv1.bias = model.encoder.model.conv_stem.bias
        new_conv2.bias = model.encoder.model.conv_stem.bias

    new_conv = nn.Sequential(
        new_conv1,
        nn.BatchNorm2d(conv1.out_channels),
        nn.SiLU(),
        new_conv2,
    )
    # заменяем веса, если количнество входных каналов не равно трем 
    if nn_in_cannels_num != 3:
        #model.encoder.conv1 = new_conv1
        #model.encoder.model.conv_stem = new_conv1
        model.encoder._conv_stem = new_conv
        #model.encoder.model.patch_embed.proj = new_conv1

    if is_multitask:
        model = MultitaskModel(model, nn_output_size=512, appl_class_num=2, surf_class_num=class_num)

    if preprocess_params['type'] == 'no':
        preprocess_layer = nn.Identity()
    elif preprocess_params['type'] == '1L':
        preprocess_layer = nn.Sequential(
            nn.Conv2d(in_channels=image_channels, out_channels=nn_in_cannels_num, kernel_size=1),
            nn.BatchNorm2d(nn_in_cannels_num)
        )
    elif preprocess_params['type'] == '2L':
        preprocess_layer = nn.Sequential(
            nn.Conv2d(in_channels=image_channels, out_channels=32, kernel_size=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=nn_in_cannels_num, kernel_size=1),
            nn.BatchNorm2d(nn_in_cannels_num),
            nn.ReLU())
    elif preprocess_params['type'] == 'SpInd':
        preprocess_layer = SpectralDiffIndexModule(channel_indices_list=channel_indices_list, channels_in_index=2, out_channels=8)
    
    if fuze_params['type'] == 'no':
        model = MultispectralNN(model, preprocess_layer)
    else:
        model = MultispectralFuseOut(model, preprocess_layer, preprocessing_out_dim=nn_in_cannels_num, fusion_type=fuze_params['type'], class_num=class_num)
    
    #train_transforms = v2.Compose([v2.Resize((160,160), antialias=True),v2.ToDtype(torch.float32, scale=True)])
    #test_transforms = v2.Compose([v2.Resize((160,160), antialias=True),v2.ToDtype(torch.float32, scale=True)])
    train_transforms = v2.Compose([v2.Resize((112,112), antialias=True),v2.ToDtype(torch.float32, scale=True)])
    test_transforms = v2.Compose([v2.Resize((112,112), antialias=True),v2.ToDtype(torch.float32, scale=True)])
    
    multitask_str = '_MT' if is_multitask else ''
    
    model_name = f'{model_name}'
    #model_name = f'{model_name}'
    if preprocess_params['type'] != 'no':
        model_name += f'-P{preprocess_params["type"]}'
    if fuze_params['type'] != 'no':
        model_name += f'-Fuz{fuze_params["type"].capitalize()}'
    #model_name = f'{model_name}pr-P2L-FuzOutAdd({nn_in_cannels_num})' + multitask_str
    return {'name': model_name, 'model': model, 'train_transforms':train_transforms, 'test_transforms':test_transforms}
    

# Создание объектов нейронной сети, датасетов, даталоадеров

In [11]:
# Путь до папки с данными (надо поменять на актуальный путь до папки с данными)
#path_to_dataset_root = r'I:\LANDCOVER_DATA\MULTISPECTRAL_SATELLITE_DATA\DATA_FOR_TRAINIG'
#path_to_dataset_root = '/home/aggr/mikhail_u/DATA/DATA_FOR_TRAINIG'
path_to_dataset_root = r'C:\Users\admin\python_programming\DATA\MULTISPECTRAL_DATA_FOR_TRAINIG_NEW'
path_to_dataset_info_csv = os.path.join(path_to_dataset_root, 'data_info_table.csv')
path_to_surface_classes_json = os.path.join(path_to_dataset_root, 'surface_classes.json')

# чтение списка имен классов поверхностей
with open(path_to_surface_classes_json) as fd:
    surface_classes_list = json.load(fd)
# чтение таблицы с информацией о каждом изображении в выборке
images_df = pd.read_csv(path_to_dataset_info_csv)

path_to_partition_json = os.path.join(path_to_dataset_root, 'dataset_partition.json')
# чтение словаря со списками квадратов, находящихся в обучающей и тестовой выборке
with open(path_to_partition_json) as fd:
    partition_dict = json.load(fd)

# заменить при перетасовке классов
applicable_surfaces_dict = {
    'UNLABELED': False,
    'buildings_territory': False,
    'natural_ground': True,
    'natural_grow': True,
    'natural_wetland': True,
    'natural_wood': True,
    'quasi_natural_grow': False,
    'transport': False,
    'water': False
}

# формирование pandas DataFrame-ов с информацией об изображениях обучающей и тестовой выборках
train_images_df = []
for train_square in partition_dict['train_squares']:
    train_images_df.append(images_df[images_df['square_id']==train_square])
train_images_df = pd.concat(train_images_df, ignore_index=True)

test_images_df = []
for test_square in partition_dict['test_squares']:
    test_images_df.append(images_df[images_df['square_id']==test_square])
test_images_df = pd.concat(test_images_df, ignore_index=True)

#train_images_df, test_images_df = train_test_split(images_df, test_size=0.3, random_state=0)

class_num = images_df['class_num'].iloc[0]

# формирование словаря, отображающейго имя класса поверхности в индекс класса
class_name2idx_dict = {n:i for i, n in enumerate(surface_classes_list)}

# вычисление распределений пикселей в классах поверхностей 
classes_pixels_distribution_df = images_df[surface_classes_list]
classes_pixels_num = classes_pixels_distribution_df.sum()
classes_weights = classes_pixels_num / classes_pixels_num.sum()
classes_weights = classes_weights[surface_classes_list].to_numpy()

# определение индексов обрабатываемых каналов мультиспектра
channel_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
#channel_indices = [1, 2, 3, 7]
#channel_indices = [1, 2, 3]
in_channels_num = len(channel_indices)
is_multitask = False
'''
# словари с настройками
deeplab_dict = {
    'model_name': 'dlv3_res50',
    'creation_function': models.segmentation.deeplabv3_resnet50,
    'weights': models.segmentation.DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1
}
fcn_dict = {
    'model_name': 'fcn_res50',
    'creation_function': models.segmentation.fcn_resnet50,
    'weights': models.segmentation.FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1
}

# создаем модель по введенным настройкам из библиотеки torchvision
model_dict = prepare_torch_model(
    model_dict=fcn_dict,
    image_channels=in_channels_num,
    nn_in_cannels_num=in_channels_num,
    class_num=class_num,
    channel_indices_list=channel_indices,
    is_multitask=is_multitask,
    preprocess_params={'type':'no'}, #'no', 1L, 2L, SpInd
    fuze_params={'type':'no'} #'no', shuffle, concat, add
    )
#model_dict  = prepare_torch_model(image_channels=in_channels_num, nn_in_cannels_num=in_channels_num, class_num=class_num, channel_indices_list=channel_indices, is_multitask=is_multitask)
#model_dict  = prepare_torch_model(image_channels=in_channels_num, nn_in_cannels_num=in_channels_num, class_num=2, channel_indices_list=channel_indices, is_multitask=is_multitask)
#model_dict  = prepare_torch_model(image_channels=in_channels_num, nn_in_cannels_num=in_channels_num, class_num=class_num, channel_indices_list=channel_indices, is_multitask=is_multitask)

# раскомментировать, если надо обучить модель из библиотеки segmentation_models_pytorch
'''
unet_dict = {
    'model_name': 'unet_effnet_d-5_strd1_1inConv',
    'creation_function': smp.Unet,
    'encoder_name': 'efficientnet-b0'
}

model_dict  = prepare_smp_model(
    model_dict=unet_dict,
    image_channels=in_channels_num,
    nn_in_cannels_num=in_channels_num,
    class_num=class_num,
    channel_indices_list=channel_indices,
    is_multitask=False,
    preprocess_params={'type':'no'}, # 'no', 1L, 2L, SpInd
    fuze_params={'type':'no'} # shuffle, concat, add
    )

# получаем основные параметры нейронной сети
model_name = model_dict['name']
model = model_dict['model']
train_transforms = model_dict['train_transforms']
test_transforms = model_dict['test_transforms']

# по умолчанию, обучаем на GPU
device = torch.device('cuda:0')
#device = torch.device('cpu')

# определение функций потерь (несколько ф-ций было взято для исследования, какая себя лучше проявит)
gamma_val = 2
criterion = nn.CrossEntropyLoss()
multitask_criterion = MultitaskLoss(nn.CrossEntropyLoss(), nn.CrossEntropyLoss())
classes_weights = torch.as_tensor(classes_weights, dtype=torch.float32, device=device)
focal_criterion = torch.hub.load(
        'adeelh/pytorch-multi-class-focal-loss',
        model='FocalLoss',
        alpha=classes_weights,
        gamma=gamma_val,
        reduction='mean',
        force_reload=False
    )

model = model.to(device)

# создаем датасеты и даталоадеры
train_dataset = SegmentationDataset(path_to_dataset_root=path_to_dataset_root, samples_df=train_images_df, channel_indices=channel_indices, transforms=train_transforms, device=device)
test_dataset = SegmentationDataset(path_to_dataset_root=path_to_dataset_root, samples_df=test_images_df,channel_indices=channel_indices, transforms=test_transforms, device=device)
#train_dataset = SegmentationDatasetApplSurf(path_to_dataset_root=path_to_dataset_root, samples_df=test_images_df, channel_indices=channel_indices, name2class_idx_dict=class_name2idx_dict, applicable_surfaces_dict=applicable_surfaces_dict, transforms=test_transforms, device=device)
#test_dataset = SegmentationDatasetApplSurf(path_to_dataset_root=path_to_dataset_root, samples_df=test_images_df, channel_indices=channel_indices, name2class_idx_dict=class_name2idx_dict, applicable_surfaces_dict=applicable_surfaces_dict, transforms=test_transforms, device=device)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16)

# тестовое чтение данных
for data, labels in train_loader:
    break
    pred = model(data)
    loss = criterion(pred, labels)

# тестовая обработка данных нейронной сетью
ret = model(data)
if is_multitask:
    img_size = list(ret[0].shape[2:])
    # печатаем выход нейронной сети
    print(ret[0].shape, ret[1].shape)
else:
    img_size = list(ret.shape[2:])
    # печатаем выход нейронной сети
    print(data.shape, ret.shape)
#

model_name

Using cache found in C:\Users\admin/.cache\torch\hub\adeelh_pytorch-multi-class-focal-loss_master


torch.Size([16, 13, 112, 112]) torch.Size([16, 11, 112, 112])


'unet_effnet_d-5_strd1_1inConv'

# Однозадачное обучение

In [12]:
epoch_num = 100
#model_name = 'FCN-150ep-11cl-150-ch-res-10-20m'
channels_str = prepare_channel_indices_str(channel_indices)

# формируем имя модели для лога
model_name = f'{model_name}-{epoch_num}ep-{class_num}cl-{img_size[0]}-ch_[{channels_str}]'
#model_name = f'{model_name}-{epoch_num}ep-2cl-{img_size[0]}-ch_[{channels_str}]'
print('#############################')
print(model_name)
print('#############################')
print()
#model_name = 'TEST'

# создаем список словарей с информацией о вычисляемых метриках с помощью multiclass confusion matrix
# см. подробнее ддокументацию к функции compute_metric_from_confusion
metrics_info_list = [
    {'name': 'iou', 'numerator': ['tp'], 'denominator': ['tp', 'fp', 'fn']},
    {'name': 'recall', 'numerator': ['tp'], 'denominator': ['tp', 'fn']},
    {'name': 'precision', 'numerator': ['tp'], 'denominator': ['tp', 'fp']}
    ]

# Создаем модуль Lightning
segmentation_module = SegmentationModule(model, criterion, metrics_info_list, class_name2idx_dict)

# задаем путь до папки с логгерами и создаем логгер, записывающий результаты в csv
path_to_saving_dir = 'saving_dir'
logger = CSVLogger(
    save_dir = path_to_saving_dir,
    name=model_name, 
    flush_logs_every_n_steps=1,
    )

# создаем объект, записывающий в чекпоинт лучшую модель
checkpoint_callback = ModelCheckpoint(
    mode="max",
    filename=model_name+"-{epoch:02d}-{val_iou_mean:.3}",
    dirpath=os.path.join(path_to_saving_dir, model_name), 
    save_top_k=1, monitor="val_iou_mean"
    )

trainer = L.Trainer(logger=logger,
        max_epochs=epoch_num, 
        callbacks=[checkpoint_callback],
        accelerator = 'gpu'
        )

trainer.fit(segmentation_module , train_loader, test_loader)

GPU available: True (cuda), used: True


#############################
unet_effnet_d-5_strd1_1inConv-100ep-11cl-112-ch_[0-12]
#############################



TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\admin\miniconda3\envs\deep_learning\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\admin\python_programming\MultispectralSegmentation\saving_dir\unet_effnet_d-5_strd1_1inConv-100ep-11cl-112-ch_[0-12] exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | MultispectralNN  | 6.3 M  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
6.3 M     Trainable params
0         Non-trainable params
6.3 M     Total params
25.060    Total estimated model params size (MB)
330       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\admin\miniconda3\envs\deep_learning\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


c:\Users\admin\miniconda3\envs\deep_learning\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Epoch 99: 100%|██████████| 88/88 [00:23<00:00,  3.81it/s, v_num=0, val_loss=1.070, val_iou_UNLABELED=0.823, val_iou_buildings_territory=0.705, val_iou_natural_ground=0.120, val_iou_natural_grow=0.116, val_iou_natural_wetland=0.353, val_iou_natural_wood=0.761, val_iou_quasi_natural_ground=0.00959, val_iou_quasi_natural_grow=0.467, val_iou_quasi_natural_wetland=0.229, val_iou_transport=0.303, val_iou_water=0.852, val_iou_mean=0.431, val_recall_UNLABELED=0.885, val_recall_buildings_territory=0.831, val_recall_natural_ground=0.213, val_recall_natural_grow=0.147, val_recall_natural_wetland=0.493, val_recall_natural_wood=0.932, val_recall_quasi_natural_ground=0.0182, val_recall_quasi_natural_grow=0.645, val_recall_quasi_natural_wetland=0.240, val_recall_transport=0.411, val_recall_water=0.898, val_recall_mean=0.519, val_precision_UNLABELED=0.922, val_precision_buildings_territory=0.823, val_precision_natural_ground=0.216, val_precision_natural_grow=0.355, val_precision_natural_wetland=0.554,

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 88/88 [00:23<00:00,  3.79it/s, v_num=0, val_loss=1.070, val_iou_UNLABELED=0.823, val_iou_buildings_territory=0.705, val_iou_natural_ground=0.120, val_iou_natural_grow=0.116, val_iou_natural_wetland=0.353, val_iou_natural_wood=0.761, val_iou_quasi_natural_ground=0.00959, val_iou_quasi_natural_grow=0.467, val_iou_quasi_natural_wetland=0.229, val_iou_transport=0.303, val_iou_water=0.852, val_iou_mean=0.431, val_recall_UNLABELED=0.885, val_recall_buildings_territory=0.831, val_recall_natural_ground=0.213, val_recall_natural_grow=0.147, val_recall_natural_wetland=0.493, val_recall_natural_wood=0.932, val_recall_quasi_natural_ground=0.0182, val_recall_quasi_natural_grow=0.645, val_recall_quasi_natural_wetland=0.240, val_recall_transport=0.411, val_recall_water=0.898, val_recall_mean=0.519, val_precision_UNLABELED=0.922, val_precision_buildings_territory=0.823, val_precision_natural_ground=0.216, val_precision_natural_grow=0.355, val_precision_natural_wetland=0.554,

# Многозадачное обучение

In [6]:
# multitask
epoch_num = 100
#model_name = 'FCN-150ep-11cl-150-ch-res-10-20m'
channels_str = prepare_channel_indices_str(channel_indices)

model_name = f'{model_name}-{epoch_num}ep-{class_num}cl-{img_size[0]}-ch_[{channels_str}]'
print('#############################')
print(model_name)
print('#############################')
print()
#model_name = 'TEST'

metrics_info_list = [
    {'name': 'iou', 'numerator': ['tp'], 'denominator': ['tp', 'fp', 'fn']},
    {'name': 'recall', 'numerator': ['tp'], 'denominator': ['tp', 'fn']},
    {'name': 'precision', 'numerator': ['tp'], 'denominator': ['tp', 'fp']}
    ]
metrics_info_list = [
    {'name': 'iou', 'numerator': ['tp'], 'denominator': ['tp', 'fp', 'fn']}
    ]

segmentation_module = MultitaskSegmentationModule(model, multitask_criterion, metrics_info_list, class_name2idx_dict, applicable_surfaces_dict)
path_to_saving_dir = 'saving_dir'
logger = CSVLogger(
    save_dir = path_to_saving_dir,
    name=model_name, 
    flush_logs_every_n_steps=1, 
    )

checkpoint_callback = ModelCheckpoint(
    mode="max",
    filename=model_name+"-{epoch:02d}-{v_ap_iou_mean:.3}-{v_cl_iou_mean:.3}-{v_iou_mean:.3}",
    dirpath=os.path.join(path_to_saving_dir, model_name), 
    save_top_k=1, monitor="v_iou_mean"
    )

trainer = L.Trainer(logger=logger,
        max_epochs=epoch_num, 
        callbacks=[checkpoint_callback],
        accelerator = 'cuda'
        )

trainer.fit(segmentation_module , train_loader, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\mokhail\miniconda3\envs\deep_learning\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\mokhail\python_programming\MultispectralSegmentation\saving_dir\fcn_res50pr-P1L(13)_MT-100ep-9cl-150-ch_[0-12] exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type           | Params | Mode 
-----------------------------------------------------
0 | model     | MultitaskModel | 35.3 M | train
1 | criterion | MultitaskLoss  | 0      | train
-----------------------------------------------------
35.3 M    Trainable params
0         Non-trainable params
35.3 M    Total params
141.394   Total estimated model params size (MB)
170       Modules in train mode
0         Modules in eval mode


#############################
fcn_res50pr-P1L(13)_MT-100ep-9cl-150-ch_[0-12]
#############################

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\mokhail\miniconda3\envs\deep_learning\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


c:\Users\mokhail\miniconda3\envs\deep_learning\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 99: 100%|██████████| 98/98 [00:24<00:00,  3.97it/s, v_num=1, val_loss=1.680, v_cl_iou_UNLABELED=0.772, v_cl_iou_buildings_territory=0.737, v_cl_iou_natural_ground=0.0843, v_cl_iou_natural_grow=0.281, v_cl_iou_natural_wetland=0.229, v_cl_iou_natural_wood=0.785, v_cl_iou_quasi_natural_grow=0.601, v_cl_iou_transport=0.0979, v_cl_iou_water=0.673, v_cl_iou_mean=0.473, v_ap_iou_non_appl=0.740, v_ap_iou_appl=0.842, v_ap_iou_mean=0.791, v_iou_mean=0.632, v_cl_recall_UNLABELED=0.887, v_cl_recall_buildings_territory=0.849, v_cl_recall_natural_ground=0.106, v_cl_recall_natural_grow=0.384, v_cl_recall_natural_wetland=0.267, v_cl_recall_natural_wood=0.947, v_cl_recall_quasi_natural_grow=0.784, v_cl_recall_transport=0.128, v_cl_recall_water=0.724, v_cl_recall_mean=0.564, v_ap_recall_non_appl=0.841, v_ap_recall_appl=0.920, v_ap_recall_mean=0.881, v_recall_mean=0.722, v_cl_precision_UNLABELED=0.856, v_cl_precision_buildings_territory=0.848, v_cl_precision_natural_ground=0.291, v_cl_precision_nat

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 98/98 [00:24<00:00,  3.94it/s, v_num=1, val_loss=1.680, v_cl_iou_UNLABELED=0.772, v_cl_iou_buildings_territory=0.737, v_cl_iou_natural_ground=0.0843, v_cl_iou_natural_grow=0.281, v_cl_iou_natural_wetland=0.229, v_cl_iou_natural_wood=0.785, v_cl_iou_quasi_natural_grow=0.601, v_cl_iou_transport=0.0979, v_cl_iou_water=0.673, v_cl_iou_mean=0.473, v_ap_iou_non_appl=0.740, v_ap_iou_appl=0.842, v_ap_iou_mean=0.791, v_iou_mean=0.632, v_cl_recall_UNLABELED=0.887, v_cl_recall_buildings_territory=0.849, v_cl_recall_natural_ground=0.106, v_cl_recall_natural_grow=0.384, v_cl_recall_natural_wetland=0.267, v_cl_recall_natural_wood=0.947, v_cl_recall_quasi_natural_grow=0.784, v_cl_recall_transport=0.128, v_cl_recall_water=0.724, v_cl_recall_mean=0.564, v_ap_recall_non_appl=0.841, v_ap_recall_appl=0.920, v_ap_recall_mean=0.881, v_recall_mean=0.722, v_cl_precision_UNLABELED=0.856, v_cl_precision_buildings_territory=0.848, v_cl_precision_natural_ground=0.291, v_cl_precision_nat

In [107]:
model.model.backbone(torch.randn(1, 3, 150, 150))['out'].shape

torch.Size([1, 2048, 19, 19])

# Черновики

In [116]:

classes_weights[surface_classes_list]


buildings_territory      0.137217
natural_ground           0.000395
natural_grow             0.114785
natural_wetland          0.066782
natural_wood             0.465326
quasi_natural_ground     0.003570
quasi_natural_grow       0.112242
quasi_natural_wetland    0.005356
quasi_natural_wood       0.000000
transport                0.029412
water                    0.027175
UNLABELED                0.037740
dtype: float64

In [66]:
input = torch.randn(1, 5, requires_grad=True)
target = torch.empty(1, dtype=torch.long).random_(5)
print(target.grad)
loss = nn.CrossEntropyLoss()
output = loss(input, target)
output.backward()
print(output)
print(target.grad)
print(input)
print(target)


None
tensor(1.6373, grad_fn=<NllLossBackward0>)
None
tensor([[-0.6013, -1.8251, -0.3468,  1.6361,  0.4586]], requires_grad=True)
tensor([4])


In [42]:
out.shape

torch.Size([2, 2])

In [107]:
class_num = 3
in_features = 1
fc = nn.Linear(in_features, class_num, bias=False)
fc.weight = nn.Parameter(torch.ones(class_num, in_features))
criterion = nn.CrossEntropyLoss(reduction='sum')
x = torch.randn(1, in_features)
t = torch.empty(1, dtype=torch.long).random_(class_num)
out = fc(x)
print(x)
print(t)
print(out)
loss = criterion(out, t)
loss.backward()
print(fc.weight.grad)


tensor([[0.5006]])
tensor([1])
tensor([[0.5006, 0.5006, 0.5006]], grad_fn=<MmBackward0>)
tensor([[ 0.1669],
        [-0.3337],
        [ 0.1669]])


In [ ]:
import os
import glob
import shutil
from tqdm import tqdm

paths_to_delete = glob.glob(r'i:\AVABOS\new_projects2\*\cut\*')
paths_to_delete = [p for p in paths_to_delete if os.path.isdir(p)]
for p in tqdm(paths_to_delete):
    shutil.rmtree(p, ignore_errors=True)

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, paths_to_images_list, transforms, device):
        '''
        path_to_dataset - путь до корневой папки с датасетом
        instance_names_list - список имен экземпляров БЕЗ РАСШИРЕНИЯ!
        transforms - аугментация изображений
        '''
        super().__init__()
        self.paths_to_images_list = paths_to_images_list
        self.transforms = transforms
        self.device = device

    def __len__(self):
        return len(self.paths_to_images_list)

    def __getitem__(self, idx):
        path_to_image = self.paths_to_images_list[idx]

        #image = torch.as_tensor(np.load(path_to_image))
        #image = np.load(path_to_image)
        image = torchvision.io.decode_image(path_to_image)
        # метки читаем как одноканальное изображение
                
        image = tv_tensors.Image(image, device=self.device)
        
        transforms_dict = {'image':image}
        transformed = self.transforms(transforms_dict)
        return transformed['image']
    
paths_to_images = glob.glob(r'i:\embedding_logo_datasets\icon645\colored_icons_final\*\*.png')
transforms = v2.Compose(
    [
        v2.Resize((256, 256), antialias=True),
        v2.ToDtype(torch.float32, scale=True),
     
     ]
    )
ds = TestDataset(paths_to_images, transforms, torch.device('cuda'))
loader = torch.utils.data.DataLoader(ds, batch_size=64, shuffle=True)
for d in tqdm(loader):
    pass

In [10]:
path = r'saving_dir\my_exp_name\version_1\metrics.csv'
pd.read_csv(path)

,epoch,step,train_iou_UNLABELED,train_iou_buildings_territory,train_iou_mean,train_iou_natural_ground,train_iou_natural_grow,train_iou_natural_wetland,train_iou_natural_wood,train_iou_quasi_natural_ground,...,val_iou_natural_grow,val_iou_natural_wetland,val_iou_natural_wood,val_iou_quasi_natural_ground,val_iou_quasi_natural_grow,val_iou_quasi_natural_wetland,val_iou_quasi_natural_wood,val_iou_transport,val_iou_water,val_loss
0,0,314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.161972,0.357346,0.734698,0.000085,0.115869,0.056907,0.0,0.056050,0.227701,1.393110
1,0,314,0.840341,0.625207,0.391350,0.0,0.283973,0.437569,0.799967,0.006817,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.483770,0.640979,0.825922,0.082659,0.653898,0.566895,0.0,0.024938,0.277133,0.568032
3,1,629,0.852058,0.649716,0.442709,0.0,0.360270,0.495965,0.833841,0.049213,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.273776,0.656812,0.766886,0.101420,0.460483,0.745244,0.0,0.072769,0.265772,1.006425
5,2,944,0.861290,0.682973,0.515124,0.0,0.454220,0.613231,0.861346,0.125224,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
